# Sudoku Constraints mit Tensoren verstehen (4×4)
Dieses notebook erklärt Schritt für Schritt den folgenden Loss-Block (Row/Col/Block/Givens) **anhand konkreter Examplee**:

- `row_sum = P.sum(dim=1)`  
- `col_sum = P.sum(dim=0)`  
- Block-Slicing + `reshape`  
- Givens-Masking + `nll_loss(log(P))`

We arbeiten mit einem **4×4 Sudoku** (Ziffern 1..4) and **2×2 Blöcken**.

In [ ]:
import torch
import torch.nn.functional as F

torch.set_printoptions(precision=3, sci_mode=False)

## 1) Was ist `P`?

`P` ist ein Wahrscheinlichkeits-Tensor mit Shape `(4,4,4)`:

- `P[i, j, k]` = Wahrscheinlichkeit, dass Zelle `(i,j)` die Ziffer `(k+1)` enthält.
- Für jede Zelle gilt: `P[i,j,:].sum() == 1`.

In [ ]:
# Ein bewusst einfaches Beispiel für P:
# - Zeile 0 ist "hart" korrekt (one-hot pro Zelle)
# - alle anderen Zellen sind zunächst uniform (0.25 pro Ziffer)

P = torch.zeros(4,4,4)

# Zeile 0: [1,2,3,4] als one-hot
P[0,0,0] = 1
P[0,1,1] = 1
P[0,2,2] = 1
P[0,3,3] = 1

# Rest: uniform unsicher
P[1:,:,:] = 0.25

# Check: jede Zelle summiert über k zu 1
cell_sums = P.sum(dim=2)
cell_sums

Wenn alles passt, ist `cell_sums` überall `1.0`.

## 2) Row-Constraint (Zeilen-Eindeutigkeit)

Für jede Zeile `i` and jede Ziffer `k` gilt:

\[
\sum_j P[i,j,k] = 1
\]

Interpretation: In einer Zeile soll jede Ziffer (1..4) **genau einmal** vorkommen.

In [ ]:
row_sum = P.sum(dim=1)  # Summe über Spalten j
row_sum

- `P` hat Shape `(4,4,4)`  
- `P.sum(dim=1)` summiert über die **Spalten** ⇒ Ergebnis Shape `(4,4)` = `(rows, digits)`

`row_sum[i,k]` sagt dir: "Wie viel Wahrscheinlichkeit steckt in Zeile i insgesamt auf Ziffer k?"

In [ ]:
L_row = ((row_sum - 1.0) ** 2).sum()
L_row

Der Row-Loss ist 0, wenn **jede** Zeile für **jede** Ziffer exakt Summe 1 erreicht.

## 3) Col-Constraint (Spalten-Eindeutigkeit)

Für jede Spalte `j` and jede Ziffer `k`:

\[
\sum_i P[i,j,k] = 1
\]

Interpretation: In einer Spalte soll jede Ziffer (1..4) **genau einmal** vorkommen.

In [ ]:
col_sum = P.sum(dim=0)  # Summe über Zeilen i
col_sum

- `P.sum(dim=0)` summiert über die **Zeilen** ⇒ Ergebnis Shape `(4,4)` = `(cols, digits)`  
`col_sum[j,k]` sagt dir: "Wie viel Wahrscheinlichkeit steckt in Spalte j insgesamt auf Ziffer k?"

In [ ]:
L_col = ((col_sum - 1.0) ** 2).sum()
L_col

## 4) Block-Constraint (2×2 Blöcke)

Für jeden 2×2 Block `b` and jede Ziffer `k` gilt:

\[
\sum_{(i,j) \in b} P[i,j,k] = 1
\]

We schneiden Blöcke via Slicing aus `P` heraus and summieren über die 4 Zellen des Blocks.

### 4.1) Einen Block anschauen (oben links)
Block oben links umfasst Zeilen 0..1 and Spalten 0..1.

In [ ]:
blk = P[0:2, 0:2, :]   # (2,2,4)
blk.shape, blk

### 4.2) Block flatten + Summe pro Ziffer

We wollen die vier Zellen des Blocks als Liste (N=4) betrachten:
- `blk.reshape(-1, 4)` macht `(2,2,4)` → `(4,4)`
- dann summieren wir über die 4 Zellen (`dim=0`) ⇒ Ergebnis `(4,)` (eine Summe pro Ziffer)

In [ ]:
blk_flat = blk.reshape(-1, 4)     # (4,4)
blk_sum  = blk_flat.sum(dim=0)    # (4,)
blk_flat, blk_sum

In [ ]:
blk_loss = ((blk_sum - 1.0) ** 2).sum()
blk_loss

### 4.3) Alle 2×2 Blöcke iterieren
Für 4×4 mit 2×2 Blöcken starten Block-Reihen bei `br = 0,2` and Block-Spalten bei `bc = 0,2`.

In [ ]:
L_blk = 0.0
for br in range(0, 4, 2):
    for bc in range(0, 4, 2):
        blk = P[br:br+2, bc:bc+2, :]       # (2,2,4)
        blk_sum = blk.reshape(-1, 4).sum(dim=0)  # (4,)
        L_blk = L_blk + ((blk_sum - 1.0) ** 2).sum()

L_blk

## 5) Givens-Constraint (harte Vorgaben)

`puzzle` enthält:
- `0` = leer
- `1..4` = vorgegebene Zahl (given)

We bauen:
- `givens_mask = puzzle > 0` (Bool-Maske)
- `givens_target = puzzle - 1` (0-basierte Klassen: 0..3)

Dann:
- sammeln wir nur die Wahrscheinlichkeiten der Given-Zellen: `given_P = P[givens_mask]` → Shape `(Ngivens, 4)`
- and die korrekten Klassenindizes: `targets = givens_target[givens_mask]` → Shape `(Ngivens,)`
- Loss: `nll_loss(log(P), targets)`

In [ ]:
puzzle = torch.tensor([
    [1, 0, 0, 4],
    [0, 0, 0, 0],
    [0, 0, 0, 0],
    [3, 0, 0, 2]
])

givens_mask = puzzle > 0
givens_target = puzzle.clamp(min=1) - 1  # 1..4 -> 0..3; 0 wird durch clamp sicher gemacht

puzzle, givens_mask, givens_target

In [ ]:
given_P = P[givens_mask]                 # (Ngivens, 4)
targets = givens_target[givens_mask]     # (Ngivens,)
given_P.shape, targets

### Warum `clamp(min=1)`?
Weil leere Felder `0` sonst zu `-1` würden (ungültiger Index).  
Wichtig: **Leere Felder werden durch `givens_mask` ohnehin ausgeschlossen** – der Wert dort ist für den Loss irrelevant.

In [ ]:
eps = 1e-9
if givens_mask.any():
    L_giv = F.nll_loss((given_P + eps).log(), targets, reduction="sum")
else:
    L_giv = P.new_tensor(0.0)

L_giv

## 6) Gesamt-Loss (Example)
Du kannst die Teile jetzt gewichten and aufsummieren.

In [ ]:
w_row, w_col, w_blk, w_giv = 1.0, 1.0, 1.0, 2.0
L_total = w_row*L_row + w_col*L_col + w_blk*L_blk + w_giv*L_giv
L_row, L_col, L_blk, L_giv, L_total

## 7) Mini-Experiment: Absichtlich einen Row-Fehler erzeugen
We machen Zeile 0 kaputt: zwei Zellen bekommen dieselbe Ziffer.

In [ ]:
P_bad = P.clone()
# Setze (0,2) auch auf Ziffer 2 (Index 1) statt 3 (Index 2)
P_bad[0,2,:] = 0
P_bad[0,2,1] = 1

row_sum_bad = P_bad.sum(dim=1)
L_row_bad = ((row_sum_bad - 1.0) ** 2).sum()
row_sum_bad[0], L_row, L_row_bad

Du siehst: Sobald eine Ziffer in der Zeile "zu viel" Wahrscheinlichkeit bekommt, steigt der Row-Loss.